In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from glob import glob
%matplotlib inline

In [2]:
sections = r"D:\Projects\Pr\2505.12\Eduvizig\Sections (waterkeringen)\geo_sections_4326_measure_join.shp"
station1 = r"D:\Projects\Pr\2505.12\Eduvizig\Sensor stations (waterstandlocaties)\stations_bak.shp"
station2 = r"D:\Projects\Pr\2505.12\Eduvizig\Sensor stations (waterstandlocaties)\stations.shp"
gdf_station1 = gpd.read_file(station1)
gdf_station2 = gpd.read_file(station2)
gdf_sections = gpd.read_file(sections)

In [3]:
station_join = pd.merge(gdf_station1, gdf_station2, left_on='name', right_on='name')
section_station_join = pd.merge(gdf_sections, station_join, left_on='stationid', right_on='id_y')[['stationid', 'id', 'min_waterl', 'max_waterl', 'threshold_', 'threshol_1', 'threshol_2']]

In [4]:
df_all = pd.DataFrame()
for row in section_station_join.iterrows():
    loc = row[1]
    loc_info = {
        'stationid':loc.stationid,
        'sectionid':loc.id,
        'min':loc['min_waterl']/100, 
        'max':loc['max_waterl']/100, 
        '0_thd': loc['threshold_']/100, 
        '1_thd': loc['threshol_1']/100, 
        '2_thd': loc['threshol_2']/100
    }
    thd_level = {
        '0_thd':0.0003,#0/3, 
        '1_thd':0.003,#1/3, 
        '2_thd':0.03 #2/3
    }
    
    if loc_info['0_thd'] == 0 and loc_info['1_thd'] == 0 and loc_info['2_thd'] == 0:
        waterlevel_divid = (loc_info['max'] - loc_info['min']) / 8
        loc_info['0_thd'] = loc_info['min'] + (5 * waterlevel_divid)
        loc_info['1_thd'] = loc_info['min'] + (6 * waterlevel_divid)        
        loc_info['2_thd'] = loc_info['min'] + (7 * waterlevel_divid)                
    
    #print(loc['name'], loc.FID_1, loc.id_1, loc_info)
    
    df = pd.DataFrame(pd.Series([
        loc_info['min'],
        loc_info['0_thd']-0.001, 
        loc_info['0_thd'],
        loc_info['1_thd']-0.001,
        loc_info['1_thd'],
        loc_info['2_thd']-0.001,
        loc_info['2_thd'], 
        loc_info['max']]), columns=['hydraulicload'])
    df['sectionid'] = int(loc_info['sectionid'])
    df['failuremechanismid'] = [[1,2]]*len(df)
    df['measureid'] = [[0,1,2,3]]*len(df)
    #df.insert(1, 'measureid', [0,1])
    df['failureprobability'] = 0.5
    

    for k in list(thd_level.keys())[::-1]:
        threshold_hload = loc_info[k]
        failure_prob = thd_level[k]
        df.loc[df.hydraulicload < threshold_hload, 'failureprobability'] = failure_prob    
    #df.plot('hydraulicload', 'failureprobability')
    df_all = pd.concat((df_all,df))

df_all = df_all[['sectionid', 'failuremechanismid', 'measureid','hydraulicload', 'failureprobability']]
df_all = df_all.explode('measureid')
df_all = df_all.explode('failuremechanismid')
df_all.drop_duplicates(inplace=True)
df_all = df_all.sort_values(['sectionid', 'failuremechanismid', 'measureid', 'hydraulicload'])

In [5]:
df_all.head(10)

,sectionid,failuremechanismid,measureid,hydraulicload,failureprobability
0,1,1,0,0.130,0.0003
1,1,1,0,4.999,0.0003
2,1,1,0,5.000,0.0030
3,1,1,0,5.999,0.0030
4,1,1,0,6.000,0.0300
5,1,1,0,6.499,0.0300
6,1,1,0,6.500,0.5000
7,1,1,0,8.130,0.5000
0,1,1,1,0.130,0.0003
1,1,1,1,4.999,0.0003


In [70]:
df_all.to_csv(r"P:\PR\3730.10\Werkzaamheden\PR3730.15_IT\Data\Eduvizig\fragility_curves_2.csv", index=False)